In [1]:
%cd ..

/Users/floydluo/Library/CloudStorage/GoogleDrive-jjluo@terpmail.umd.edu/My Drive/0-Research-Project/MedStar/MS_CODE/FieldNN


# Embed

## Get Input Tensor

In [2]:
import os
from fieldnn.dataset import RFGDataset, my_collate_fn
from torch.utils.data import DataLoader

Tensor_folder = 'data/ProcData/FldGrnTensor/'
recfldgrn_list = ['P@PatEcInfo-InfoGrn',  'P-EC-PNSect@AllText-TknzGrn']
full_recfldgrn_list = ['B-' + i for i in recfldgrn_list]

# from the get_grain_fn to get the Elig_Set.
Elig_Set = ['P4', 'P5', 'P6', 'P7']

dataset = RFGDataset(Tensor_folder, recfldgrn_list, Elig_Set, RecRootID = 'PID')
print(len(dataset), '<---- dataset')
dataloader = DataLoader(dataset, batch_size = 4, shuffle = True, collate_fn = my_collate_fn)
print(len(dataloader), '<---- dataset')


for idx, batch in enumerate(dataloader):
    # print(f'\n------ {idx}')
    batch_rfg, batch_y = batch
    for k, v in batch_rfg.items(): print(k, v.shape)
    break
    # for k, v in batch_rfg.items(): print(k, v.shape)
    # print(batch_y.shape)

4 <---- dataset
1 <---- dataset
B-P@PatEcInfo-InfoGrn_wgt torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_tknidx torch.Size([4, 43])
B-P@PatEcInfo-InfoGrn_fldidx torch.Size([4, 43])
B-P-EC-PNSect@AllText-TknzGrn_wgt torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_tknidx torch.Size([4, 23, 14, 221])
B-P-EC-PNSect@AllText-TknzGrn_fldidx torch.Size([4, 23, 14, 221])


In [3]:
# Prepare the Input
RECFLD_TO_TENSOR = {}
for full_recfldgrn in full_recfldgrn_list:
    RECFLD_TO_TENSOR[full_recfldgrn] = {k: v for k, v in batch_rfg.items() if full_recfldgrn in k}
    
[i for i in RECFLD_TO_TENSOR]

['B-P@PatEcInfo-InfoGrn', 'B-P-EC-PNSect@AllText-TknzGrn']

In [4]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit

from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################


default_SubUnitName = 'E'
fldgrn_folder = 'data/ProcData/FldGrnInfo'
learner_default_dict = {} # To update it in the future. 


df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_SubUnitName)

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s



s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,E,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,[expander-CateEmbed],"[{'full_recfldgrn': 'B-P@PatEcInfo-InfoGrn', '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,E,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,[expander-LLMEmbed],[{'full_recfldgrn': 'B-P-EC-PNSect@AllText-Tkn...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


In [5]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


## Get df_SubUnit

### Get Original df_SubUnit

In [6]:
from fieldnn.dataflowfn.embedflowfn import get_EmbeddingBlock_SubUnit
from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

############################
embed_size = 128
process = {'activator': 'gelu',
           'dropout': {'p': 0.5, 'inplace': False},
           'layernorm': {'eps': 1e-05, 'elementwise_affine': True}}
############################

############################### NEW THING: replace E with EL
default_SubUnitName = 'EL'
fldgrn_folder = 'data/ProcData/FldGrnInfo'


import pandas as pd
def get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_R_subunit_name = 'E'):
    SubUnit_List = []
    for input_name in full_recfldgrn_list:
        d = {}
        d['SubUnitName'] = default_R_subunit_name
        
        recfldgrn = '-'.join(input_name.split('-')[-2:])
        output_layerid = len(input_name.split('-'))
        output_name = input_name.split('Grn')[0]
        
        d['input_names'] = [input_name]
        d['output_name'] = output_name
        
        d['output_layerid'] = output_layerid
        
        SubUnit_List.append(d)
        
    df_SubUnit = pd.DataFrame(SubUnit_List)
    
    return df_SubUnit


default_R_subunit_name = 'EL'
df_SubUnit = get_EmbeddingBlock_SubUnit(full_recfldgrn_list, default_R_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid
0,EL,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3
1,EL,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5


### Get SubUnit Name's NN List

In [7]:
def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            assert len(input_names) == 1
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)
df_SubUnit['SubUnit_BasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List
0,EL,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,"[expander-CateEmbed, learner-TFM]"
1,EL,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,"[expander-LLMEmbed, learner-TFM]"


### Get Default BasicNN Config List

In [8]:
from fieldnn.dataflowfn.embedflowfn import get_Default_ExpanderNNPara

def get_SubUnit_Default_NNPara_List(SubUnit_BasicNN_List, SubUnit_input_names, 
                                    fldgrn_folder, learner_default_dict):
    
    SubUnit_DefaultBasicNN_List = []
    for basic_nn_idx, nn_type_nn_name in enumerate(SubUnit_BasicNN_List):
        nn_type, nn_name = nn_type_nn_name.split('-')
        
        if basic_nn_idx == 0:
            # the first BasicNN in the SubUnit
            # expander and merger will only be here.
            # this assigments only works for the first iteration
            # let's make a village rule: only E, R, M can be the first. 
            assert nn_type in ['expander', 'reducer', 'merger']
        
            if nn_type == 'expander':
                input_names_nnlvl = SubUnit_input_names 
                assert len(input_names_nnlvl) == 1
                full_recfldgrn = input_names_nnlvl[0]
                default_para = get_Default_ExpanderNNPara(full_recfldgrn, fldgrn_folder)
                
            else:
                default_para = {}
            
        else:
            assert nn_type in ['reducer', 'learner']
            if nn_type == 'learner':
                default_para = learner_default_dict[nn_name] # TODO
            else:
                default_para = {}
        
        SubUnit_DefaultBasicNN_List.append(default_para)    
        
    return SubUnit_DefaultBasicNN_List
    


###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################
    
    
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,EL,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,"[expander-CateEmbed, learner-TFM]","[{'full_recfldgrn': 'B-P@PatEcInfo-InfoGrn', '..."
1,EL,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,"[expander-LLMEmbed, learner-TFM]",[{'full_recfldgrn': 'B-P-EC-PNSect@AllText-Tkn...


### Get BasicNN Config List

In [9]:
from fieldnn.configfn.expanderfn import get_expander_para
from fieldnn.configfn.mergerfn import get_merger_para
from fieldnn.configfn.reducerfn import get_reducer_para
from fieldnn.configfn.learnerfn import get_learner_para



def generate_BasicNN_Config(nn_type_nn_name, 
                            input_names_nnlvl, 
                            default_nnpara, 
                            embed_size, 
                            process):
    '''
        please notince here, this function is not the final version yet.
    '''
    nn_type, nn_name = nn_type_nn_name.split('-')

    if nn_type == 'expander':
        
        assert len(input_names_nnlvl) == 1
        # fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].split('Grn')[0]
        
        # Get the input_size and output_size
        input_size = None
        output_size = embed_size 
        
        # Get the postprocess
        postprocess = process
        
        # Derive the para
        full_recfldgrn = default_nnpara['full_recfldgrn']
        Info = default_nnpara['Info']
        # para = get_expander_para(nn_name, default_nnpara, embed_size, vocab_tokenizer, init, postprocess)
        para = get_expander_para(full_recfldgrn, Info, embed_size, postprocess)

    elif nn_type == 'reducer': 
        
        assert len(input_names_nnlvl) == 1
        fld = input_names_nnlvl[0].split('-')[-1]
        # Get the output_name_nnlvl
        output_name_nnlvl = input_names_nnlvl[0].replace('-' + fld, '@' + fld)

        # Get the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size

        # Get the output_size
        if nn_name.lower() == 'concat':
            # this types of merger only happened in the R in MLRL. 
            # you can skip this part safely if you haven't encountered M. 
            fld_childflds = input_names_nnlvl[0].split('-')[-1]
            assert '@' in fld_childflds
            childflds = fld_childflds.split('@')[-1]
            childflds = childflds.split('&')
            output_size = len(childflds) * embed_size
        else:
            # usually the most case. 
            output_size = embed_size 

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_reducer_para(nn_name, default_nnpara, input_size, output_size, postprocess)  
        
    elif nn_type == 'merger':
        # generate the output name
        
        assert len(input_names_nnlvl) > 1
        # input_names_nnlvl: ['B-P-EC-A1C@DT', 'B-P-EC-A1C@V']
        
        childflds = [i.split('-')[-1] for i in input_names_nnlvl]
        # childflds: ['A1C@DT', 'A1C@V']
        
        fld_childflds = '&'.join([i.split('@')[-1] for i in childflds])
        # fld_childflds: DT&V
        
        output_prefix = input_names_nnlvl[0].replace('@' + childflds[0].split('@')[-1], '')
        # output_prefix: B-P-EC-A1C
        
        output_name_nnlvl = output_prefix + '-' + fld_childflds
        # output_name_nnlvl: B-P-EC-A1C-DT&V
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_merger_para(nn_name, default_nnpara, input_size, output_size, postprocess) 
        
        
    elif nn_type == 'learner':
        # generate the output name
        assert len(input_names_nnlvl) == 1
        output_name_nnlvl = input_names_nnlvl[0] # just the same name as before
        
        # Prepare the para for the NN layer
        nn_para = default_nnpara # this will be updated.

        # Get the input_size
        input_size = embed_size
        output_size = embed_size

        # Get the postprocess
        postprocess = process

        # Derive the para
        para = get_learner_para(nn_name, default_nnpara, input_size, output_size, postprocess)
        
        
    else:
        raise ValueError(f'nn_type {nn_type} is not available yet')

    Basic_Config = {'input_names_nnlvl': input_names_nnlvl, 
                    'output_name_nnlvl': output_name_nnlvl, 
                    f'{nn_type}_para': para}
    
    return Basic_Config

In [10]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,EL,[B-P@PatEcInfo-InfoGrn],B-P@PatEcInfo-Info,3,"[expander-CateEmbed, learner-TFM]","[{'full_recfldgrn': 'B-P@PatEcInfo-InfoGrn', '...","[{'nn_type_nn_name': 'expander-CateEmbed', 'Ba..."
1,EL,[B-P-EC-PNSect@AllText-TknzGrn],B-P-EC-PNSect@AllText-Tknz,5,"[expander-LLMEmbed, learner-TFM]",[{'full_recfldgrn': 'B-P-EC-PNSect@AllText-Tkn...,"[{'nn_type_nn_name': 'expander-LLMEmbed', 'Bas..."


## Embed Block

In [11]:
from fieldnn.module.embedblock import EmbedBlockLayer

EmbedBlock = EmbedBlockLayer(df_SubUnit)

RECFLD_TO_EMBEDTESNOR = EmbedBlock(RECFLD_TO_TENSOR)

for k, v in RECFLD_TO_EMBEDTESNOR.items():
    print(k, v['info'].shape)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


# Repr

## Get Input Tensor

In [12]:
for full_recfld, info_dict in RECFLD_TO_EMBEDTESNOR.items():
    print(full_recfld, info_dict['info'].shape)

B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])


## Get df_SubUnit
### Get Original df_SubUnit 

In [13]:
from fieldnn.dataflowfn.reprflowfn import get_Repr_dataflow_table
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow
from fieldnn.dataflowfn.reprflowfn import update_df_Repr_dataflow_completename
from fieldnn.dataflowfn.reprflowfn import get_Repr_SubUnit_List

In [14]:
full_recfldgrn_list = [i for i in RECFLD_TO_EMBEDTESNOR]
full_recfldgrn_list

['B-P@PatEcInfo-Info', 'B-P-EC-PNSect@AllText-Tknz']

In [15]:
df_dataflow = get_Repr_dataflow_table(full_recfldgrn_list)
# df_dataflow

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'ReducerOnly')
# df_dataflow_new

df_dataflow_new = update_df_Repr_dataflow(df_dataflow, style = 'Reducer&Merger')
# df_dataflow_new

In [16]:

def get_Repr_SubUnit_List(df_dataflow, 
                          default_R_subunit_name = 'RL', 
                          default_MR_subunit_name = 'MRL', # or 'MLRL'
                         ):
    layeridx_list = list(df_dataflow.columns)
    
    SubUnit_List = []
    for idx in range(len(layeridx_list) - 1):
        A_layerid = layeridx_list[idx]
        B_layerid = layeridx_list[idx + 1]
        # print(A_layerid, B_layerid)

        A_tensors = df_dataflow[A_layerid]
        A_tensors = A_tensors[-A_tensors.isna()].to_dict()

        B_tensors = df_dataflow[B_layerid]
        B_tensors = B_tensors[-B_tensors.isna()].to_dict()

        # print(f'\nFrom Layer {A_layerid} to {B_layerid}:')

        # from A tensor to B tensor, we have the Reducer NNs.
        # also notice that some 

        # print('A', A_tensors)
        for k in A_tensors:
            if k not in B_tensors: continue

            input_name = A_tensors[k]
            output_name = B_tensors[k]

            if pd.isna(output_name) == True: continue # pass it. 

            d = {}
            d['SubUnitName'] = default_R_subunit_name
            d['input_names'] = [input_name]
            d['output_name'] = output_name
            
            d['input_layerid'] = A_layerid
            d['output_layerid'] = B_layerid
        
        
            SubUnit_List.append(d)


        # from B tensor: potential there are some Merger NNs. 
        # print('B', B_tensors)
        # check whether these is a '(Merger') in the key
        merger_tensors = [v for k, v in B_tensors.items() if '(Merge)' in k and '@' not in v]
        # print('B-merger_tensors', merger_tensors)

        for output_tensor in merger_tensors:
            input_tensors = [i for k, i in B_tensors.items() if output_tensor + '@' in i]
            # print(output_tensor, ':', input_tensors)

            d = {}
            d['SubUnitName'] = default_MR_subunit_name
            d['input_names'] = input_tensors
            d['output_name'] = output_tensor
            
            d['input_layerid'] = B_layerid
            d['output_layerid'] = B_layerid
        
            SubUnit_List.append(d)
            
    df_SubUnit = pd.DataFrame(SubUnit_List)
    return df_SubUnit



In [17]:
df_dataflow = df_dataflow_new.copy()
default_R_subunit_name = 'RL' # 'RL'
default_MR_subunit_name = 'MRL' # 'MLRL', 'MRL', 'MR'
df_SubUnit = get_Repr_SubUnit_List(df_dataflow, default_R_subunit_name, default_MR_subunit_name)
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid
0,RL,[B-P-EC-PNSect@AllText-Tknz],B-P-EC-PNSect,5,4
1,RL,[B-P-EC-PNSect],B-P-EC,4,3
2,RL,[B-P@PatEcInfo-Info],B-P@PatEcInfo,3,2
3,RL,[B-P-EC],B-P@EC,3,2
4,MRL,"[B-P@PatEcInfo, B-P@EC]",B-P,2,2


### get SubUnit Name's NN List


Example
* E --> [expander-LLM]
* MR --> [merger-Merge, reducer-ReduceMax]


In [18]:
# from fieldnn.dataflowfn.baseflowfn import mapping_SubUnitName_to_SubUnitNNList

def mapping_SubUnitName_to_SubUnitNNList(SubUnitName, input_names, 
                                         default_BasicNNtype_To_NNName):
    SubUnitNNList = []
    for idx, NAME in enumerate(SubUnitName):
        if NAME == 'E':
            nn_type = 'expander'
            input_name = input_names[0]
            
            assert 'Grn' in input_name
            assert len(input_names) == 1
            
            if 'Tknz' in input_name:
                nn_name = 'LLMEmbed'
            elif '_wgt' in input_name:
                nn_name = 'NumeEmbed'
            else:
                nn_name = 'CateEmbed'
            
        elif NAME == 'R':
            if idx == 0: assert len(input_names) == 1
            nn_type = 'reducer'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'M':
            nn_type = 'merger'
            nn_name = default_BasicNNtype_To_NNName[nn_type]
            
        elif NAME == 'L':
            # print(input_names)
            assert idx != 0
            # assert len(input_names) == 1; not necessary, L can follow M
            nn_type = 'learner'
            input_name = input_names[0]
            layers_num = len(input_name.split('-'))
            if layers_num >= 3:
                nn_name = 'TFM'
            elif layers_num == 2:
                nn_name = 'Linear'
            else:
                raise ValueError(f'incorrect layers_num {layers_num}')
            
        else:
            raise ValueError(f'The BasicNN is not correct {NAME}')
        
        SubUnitNNList.append(nn_type + '-' + nn_name)
    return SubUnitNNList

In [19]:
############################################# Hyperparameters
default_BasicNNtype_To_NNName = {
    'expander': None, # will be updated according to the Grn Type
    'reducer': 'Max',
    'merger': 'Merger',
    'learner': None, # TODO: ignore this currently
    
}
#############################################

s = df_SubUnit.apply(lambda x: mapping_SubUnitName_to_SubUnitNNList(x['SubUnitName'], 
                                                                    x['input_names'],
                                                                    default_BasicNNtype_To_NNName), 
                    axis = 1)


df_SubUnit['SubUnit_BasicNN_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List
0,RL,[B-P-EC-PNSect@AllText-Tknz],B-P-EC-PNSect,5,4,"[reducer-Max, learner-TFM]"
1,RL,[B-P-EC-PNSect],B-P-EC,4,3,"[reducer-Max, learner-TFM]"
2,RL,[B-P@PatEcInfo-Info],B-P@PatEcInfo,3,2,"[reducer-Max, learner-TFM]"
3,RL,[B-P-EC],B-P@EC,3,2,"[reducer-Max, learner-TFM]"
4,MRL,"[B-P@PatEcInfo, B-P@EC]",B-P,2,2,"[merger-Merger, reducer-Max, learner-Linear]"


### Get Default BasicNN Config List

In [20]:

    

from fieldnn.dataflowfn.baseflowfn import get_SubUnit_Default_NNPara_List

###########################################################
psn_embedprocess = {
    # 'activator': 'gelu',
    # 'dropout': {'p': 0.5, 'inplace': False},
    # 'layernorm': {'eps': 1e-05, 'elementwise_affine': True}
}

learner_default_dict = {
    'TFM': {'psn_max': 512, 
            'psn_embedprocess': psn_embedprocess},
    'Linear': {'initrange': 0.1},   
}
###########################################################
    
    
s = df_SubUnit.apply(lambda x: get_SubUnit_Default_NNPara_List(x['SubUnit_BasicNN_List'], 
                                                               x['input_names'],
                                                               fldgrn_folder, 
                                                               learner_default_dict), axis = 1)

df_SubUnit['SubUnit_DefaultBasicNN_List'] = s

df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List
0,RL,[B-P-EC-PNSect@AllText-Tknz],B-P-EC-PNSect,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
1,RL,[B-P-EC-PNSect],B-P-EC,4,3,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
2,RL,[B-P@PatEcInfo-Info],B-P@PatEcInfo,3,2,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
3,RL,[B-P-EC],B-P@EC,3,2,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]"
4,MRL,"[B-P@PatEcInfo, B-P@EC]",B-P,2,2,"[merger-Merger, reducer-Max, learner-Linear]","[{}, {}, {'initrange': 0.1}]"


### Get BasicNN Config List

In [21]:
from fieldnn.dataflowfn.baseflowfn import get_SubUnit_BasicNN_Config_List

In [22]:
s = df_SubUnit.apply(lambda x: get_SubUnit_BasicNN_Config_List(x['SubUnit_BasicNN_List'], 
                                                               x['SubUnit_DefaultBasicNN_List'], 
                                                               x['input_names'], 
                                                               x['output_name'], 
                                                                embed_size, 
                                                                process, 
                                                               ), axis = 1)

df_SubUnit['SubUnit_BasicNN_Config_List'] = s
df_SubUnit

,SubUnitName,input_names,output_name,input_layerid,output_layerid,SubUnit_BasicNN_List,SubUnit_DefaultBasicNN_List,SubUnit_BasicNN_Config_List
0,RL,[B-P-EC-PNSect@AllText-Tknz],B-P-EC-PNSect,5,4,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
1,RL,[B-P-EC-PNSect],B-P-EC,4,3,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
2,RL,[B-P@PatEcInfo-Info],B-P@PatEcInfo,3,2,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
3,RL,[B-P-EC],B-P@EC,3,2,"[reducer-Max, learner-TFM]","[{}, {'psn_max': 512, 'psn_embedprocess': {}}]","[{'nn_type_nn_name': 'reducer-Max', 'Basic_Con..."
4,MRL,"[B-P@PatEcInfo, B-P@EC]",B-P,2,2,"[merger-Merger, reducer-Max, learner-Linear]","[{}, {}, {'initrange': 0.1}]","[{'nn_type_nn_name': 'merger-Merger', 'Basic_C..."


## Repr Block

In [23]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@PatEcInfo-Info', 'B-P-EC-PNSect@AllText-Tknz']

In [24]:
fld_updates_dict = {}
for i in RECFLD_TO_EMBEDTESNOR:
    layernum = len(i.split('-'))
    fld = i.split('-')[-1]
    if '@' in fld:
        # print(fld)
        neat_i = '-'.join(i.split('-')[:-1]) + '-' + fld.split('@')[0]
        # print(neat_i)
        same_neat_list = [t for t in RECFLD_TO_EMBEDTESNOR if neat_i + '@' in t]
        # print(same_neat_list)
        if len(same_neat_list) == 1: # itself
            # RECFLD_TO_EMBEDTESNOR[]
            fld_updates_dict[i] = neat_i
            
for old, new in fld_updates_dict.items():
    RECFLD_TO_EMBEDTESNOR[new] = RECFLD_TO_EMBEDTESNOR.pop(old)
    
    
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@PatEcInfo-Info', 'B-P-EC-PNSect@AllText-Tknz']

In [25]:
# from fieldnn.module.reprblock import ReprBlockLayer

import torch
from fieldnn.module.subunit import SubUnit_Layer


class ReprBlockLayer(torch.nn.Module):
    
    def __init__(self, df_SubUnit):
        super(ReprBlockLayer, self).__init__()
        self.df_SubUnit = df_SubUnit
        self.SubUnitDict = torch.nn.ModuleDict()
        
        for idx, SubUnit_info in df_SubUnit.iterrows():
            output_name = SubUnit_info['output_name']
            SubUnitLayer = SubUnit_Layer(SubUnit_info)
            self.SubUnitDict[output_name] = SubUnitLayer

    def forward(self, RECFLD_TO_EMBEDTESNOR):
        
        OUTPUT_TO_TENSOR = RECFLD_TO_EMBEDTESNOR.copy()
        
        for output_name, SubUnitLayer in self.SubUnitDict.items():
            input_names = SubUnitLayer.SubUnit_input_names
            # print(input_names)
            # for i in input_names:
            #     print(i, OUTPUT_TO_TENSOR[i]['info'].shape)
            SubUnit_output_name, info_dict = SubUnitLayer(input_names, OUTPUT_TO_TENSOR)
            
            assert output_name == SubUnit_output_name
            OUTPUT_TO_TENSOR[SubUnit_output_name] = info_dict
        
        return OUTPUT_TO_TENSOR
    

In [26]:
[i for i in RECFLD_TO_EMBEDTESNOR]

['B-P@PatEcInfo-Info', 'B-P-EC-PNSect@AllText-Tknz']

In [27]:


ReprBlock = ReprBlockLayer(df_SubUnit)

OUTPUT_TO_TESNOR = ReprBlock(RECFLD_TO_EMBEDTESNOR)

for k, v in OUTPUT_TO_TESNOR.items():
    print(k, v['info'].shape)

B-P@PatEcInfo-Info torch.Size([4, 43, 128])
B-P-EC-PNSect@AllText-Tknz torch.Size([4, 23, 14, 221, 128])
B-P-EC-PNSect torch.Size([4, 23, 14, 128])
B-P-EC torch.Size([4, 23, 128])
B-P@PatEcInfo torch.Size([4, 128])
B-P@EC torch.Size([4, 128])
B-P torch.Size([4, 128])
